In [3]:
import os
import sys

sys.path.append("../")
print(os.getcwd())
print(sys.path)
from segmentation_scripts.filter_records import *

/Users/zleblanc/Informing-The-Third-World/periodical_augmentation/periodical-issue-segmentation/notebooks
['/Users/zleblanc/Informing-The-Third-World/periodical_augmentation/periodical-issue-segmentation/notebooks', '/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/zleblanc/.virtualenvs/htrc-periodicals-venv/lib/python3.9/site-packages', '../', '../']


In [4]:
import pandas as pd

df = pd.read_csv("../datasets/mapping_files/directory_annotation_metadata_mapping.csv")

In [5]:
df

,annotation_file,metadata_file,magazine_name,local_dir,file_name,fuzzy_ratio,final_dir
0,solidarity_annotated_dataset.csv,solidarity_002799178.csv,solidarity,solidarity_aapso,solidarity,NaN,/solidarity_aapso
1,afro_asian_peoples_annotated_dataset.csv,afro_asian_peoples_006064527.csv,afro_asian_peoples,afro_asian_peoples,afro_asian_peoples,100.0,/afro_asian_peoples
2,liberator_annotated_dataset.csv,liberator_102427627.csv,liberator,liberator,liberator,100.0,/liberator
3,afro_asian_and_world_affairs_annotated_dataset...,afro_asian_and_world_affairs_000495647.csv,afro_asian_and_world_affairs,afro_asian_and_world_affairs,afro_asian_and_world_affairs,100.0,/afro_asian_and_world_affairs
4,afro_asian_bulletin_annotated_dataset.csv,afro_asian_bulletin_000055797.csv,afro_asian_bulletin,afro_asian_bulletin,afro_asian_bulletin,100.0,/afro_asian_bulletin
5,arab_observer_annotated_dataset.csv,arab_observer_000679918_006064523.csv,arab_observer,arab_observer_and_the_scribe,arab_observer,NaN,/arab_observer_and_the_scribe
6,arab_observer_annotated_dataset.csv,arab_observer_000679918_006064523.csv,arab_observer,arab_observer_and_the_scribe,arab_observer,NaN,/arab_observer_and_the_scribe
7,lotus_annotated_dataset.csv,lotus_000534608.csv,lotus,lotus,lotus,100.0,/lotus
8,scribe_annotated_dataset.csv,scribe_000507426.csv,scribe,scribe,scribe,100.0,/scribe
9,tricontinental_annotated_dataset.csv,tricontinental_010744668.csv,tricontinental,tricontinental,tricontinental,100.0,/tricontinental


In [2]:
os.path.exists("/Users/zleblanc/Informing-The-Third-World/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets")

True

In [6]:
import os

# Define the relative path from the current working directory
relative_path = os.path.join("..", "periodical-collection-curation", "HathiTrust-pcc-datasets", "datasets", "ht_ef_datasets")

# Compute the absolute path
absolute_path = os.path.abspath(relative_path)

# Check if the path exists
path_exists = os.path.exists(absolute_path)

print("Relative path:", relative_path)
print("Absolute path:", absolute_path)
print("Path exists:", path_exists)

Relative path: ../periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets
Absolute path: /Users/zleblanc/Informing-The-Third-World/periodical_augmentation/periodical-issue-segmentation/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets
Path exists: False


In [7]:
os.path.exists("../datasets")

True

In [16]:
relative_extracted_features_path = os.path.join("..", "..", "..", "periodical-collection-curation", "HathiTrust-pcc-datasets", "datasets", "ht_ef_datasets")
print(os.path.exists(relative_extracted_features_path))

True


In [19]:
dir_list = [subdir.replace(relative_extracted_features_path, '') for subdir, _, _ in os.walk(relative_extracted_features_path)]

In [21]:
# subset to only paths with one level of depth
subset_dir_list = [d for d in dir_list if d.count(os.sep) == 1]

In [22]:
subset_dir_list

['/liberator',
 '/spearhead',
 '/presence_africaine',
 '/arab_observer_and_the_scribe',
 "/bulletin_d’information__bureau_d'information_du_gouvernement_révolutionnaire_provisoire_de_la_république_du_sud_viet-nam_à_paris",
 '/news_from_xinhua_news_agency,_chinadaily_bulletin',
 '/oau_bulletin',
 '/black_news',
 '/lotus',
 '/tricontinental',
 '/afro_asian_and_world_affairs',
 '/afro_asian_peoples',
 '/la_documentation_arabe_',
 '/nahdat_ifriqiyah_african_renaissance',
 '/freedomways',
 '/direct_from_cuba',
 '/lsm_news__liberation_support_movement,_information_center',
 '/negro_digest',
 '/black_world',
 '/asia_and_africa_today',
 '/afro_asian_women',
 '/solidarity_swapo',
 '/afrique_documents',
 '/mozambican_revolution',
 '/al-nashrah_al-ifrīqīyah_al-āsyawīyah',
 '/solidarity_aapso',
 '/afro_asian_bulletin',
 '/im_information_bulletin__mozambique_information_agency',
 '/révolution_africaine']